#  Преобразование текстов в последовательность индексов токенов. Torchtext.

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/text/stable/
* https://pytorch.org/text/stable/vocab.html
* https://pytorch.org/text/stable/transforms.html

## Задачи для совместного разбора

1\. Рассмотрите основные шаги по преобразованию текста в последовательность индексов токенов.

In [ ]:
corpus = [
    "Студенты усердно занимаются стремясь получить знания и достичь успеха",
    "Студенты активно участвуют в общественной жизни университета"
]

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
corpus_t = [word_tokenize(w.lower()) for w in corpus]

In [ ]:
words = set()
words.update(corpus_t[0])
words.update(corpus_t[1])
words = list(words)

In [ ]:
words

['активно',
 'в',
 'получить',
 'участвуют',
 'стремясь',
 'жизни',
 'достичь',
 'и',
 'студенты',
 'усердно',
 'общественной',
 'успеха',
 'знания',
 'занимаются',
 'университета']

In [ ]:
itos = words
stoi = {v: idx for idx, v in enumerate(words)}

In [ ]:
corpus_i = [[stoi[t] for t in tokens] for tokens in corpus_t]
corpus_i

[[8, 9, 13, 4, 2, 12, 7, 6, 11], [8, 0, 3, 1, 10, 5, 14]]

In [ ]:
import torch as th

In [ ]:
corpus_i[1].extend([-1, -1]) # padding
th.tensor(corpus_i)

tensor([[ 8,  9, 13,  4,  2, 12,  7,  6, 11],
        [ 8,  0,  3,  1, 10,  5, 14, -1, -1]])

2\. Рассмотрите процесс создания `Vocab` из `torchtext`.

In [ ]:
import torchtext

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
vocab = build_vocab_from_iterator(corpus_t)
vocab.lookup_indices('занимаются')

3\. Примените преобразование `AddToken` из пакета `torchtext`.

In [ ]:
from torchtext.transforms import AddToken

In [ ]:
add = AddToken(token=999, begin=False)
add(corpus_i)

[[8, 9, 13, 4, 2, 12, 7, 6, 11, 999], [8, 0, 3, 1, 10, 5, 14, -1, -1, 999]]

## Задачи для самостоятельного решения

In [1]:
import pandas as pd
import numpy as np
import torch as th
import torchtext
from torchtext.vocab import build_vocab_from_iterator
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<p class="task" id="1"></p>

1\. Опишите класс `Vocab`. При создании объекта `Vocab` в конструктор передается набор текстов, предварительно разбитых на токены. Объект должен позволять:
* по токену получить его уникальный индекс (в случае отсутствия токена в словаре вернуть 1)
* по индексу токена получить сам токен (в случае отсутствия токена в словаре вернуть `<UNK>`)

Первые 4 индекса зарезервированы под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`.
    
Создайте `Vocab` на основе списка `corpus` и закодируйте каждый токен в предложениях, используя `Vocab`. Выведите полученный результат на экран.

- [x] Проверено на семинаре

In [48]:
class Vocab:
    def __init__(self, data):
        self.words = set()
        for tokens in data:
          self.words.update(tokens)
        self.words = ['<PAD>', '<UNK>', '<SOS>', '<EOS>'] + list(self.words)
        self.dct = {v: idx for idx, v in enumerate(self.words)}

    def itos(self, idx):
        """Возвращает токен по индексу"""
        try:
          return self.words[idx]
        except IndexError:
          return '<UNK>'

    def stoi(self, s):
        """Возвращает индекс токена"""
        return self.dct.get(s, 1)

In [ ]:
corpus = [
    "Маленький котенок игриво прыгает за шариком",
    "Пушистый котик мурлыкает, лежа на солнышке",
    "Котенок любопытно нюхает цветы в саду",
    "Котик ловко лазает по дереву, исследуя окружающий мир",
    "Спящий котик мило моргает своими яркими глазками",
]
corpus_t = [word_tokenize(text.lower()) for text in corpus]
vcb = Vocab(corpus_t)
vcb.itos(3), vcb.itos(10), vcb.stoi('саду'), vcb.stoi('jgfdk')

('<PAD>', 'мило', 31, 1)

In [ ]:
corpus_enc = [[vcb.stoi(w) for w in text] for text in corpus_t]
corpus_enc

[[30, 25, 16, 23, 13, 17],
 [5, 6, 29, 4, 27, 9, 24],
 [25, 32, 8, 26, 21, 31],
 [6, 28, 12, 15, 14, 4, 7, 11, 20],
 [33, 6, 10, 34, 19, 18, 22]]

<p class="task" id="2"></p>

2\. Создайте класс `NewsDataset` на основе данных из файла `news.csv`. Реализуйте метод `__getitem__` таким образом, чтобы он возвращал набор индексов токенов для заголовка новости (или новостей, если используются срезы) и метки классов для этих новостей. Для кодирования текстов используйте собственную реализацию `Vocab`. Там, где это возможно, возвращайте результат в виде тензора, а не списка.

Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [ ]:
df = pd.read_csv('news.csv')

In [45]:
class NewsDataset:
  def __init__(self, data):
    self.data = data.values
    self.vcb = Vocab([word_tokenize(text.lower()) for text in self.data[:, 0]])

  def __getitem__(self, idx):
    if isinstance(idx, slice):
        res = self.data[idx.start:idx.stop:idx.step]
        enc_data = [[self.vcb.stoi(w) for w in word_tokenize(text)] for text in res[:, 0]]
        return enc_data, res[:, 1]
    else:
        res = self.data[idx]
        tokens = word_tokenize(res[0])
        enc_data = [[self.vcb.stoi(w) for w in tokens]]
        return enc_data, res[1]

In [62]:
ND = NewsDataset(df)
ND[0:3], ND[0]

(([[1860, 1482, 3704, 3817, 4105, 953],
   [1718, 3608, 154, 3962, 5686, 3410, 1999, 1934, 683],
   [1860, 4557, 1076, 4221, 5714, 3410, 1999, 1139, 2736, 2875]],
  array([0, 1, 0], dtype=object)),
 ([[1860, 1482, 3704, 3817, 4105, 953]], 0))

<p class="task" id="3"></p>

3\. Реализуйте преобразование `Truncate`, которое обрезает каждый текст в батче до `n` символов. Создайте версию `NewsDataset` с обрезкой предложений до 5 токенов. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [46]:
class Truncate(NewsDataset):
  def __init__(self, data, n):
    super().__init__(data)
    self.n = n

  def __getitem__(self, idx):
    data, labels = super().__getitem__(idx)
    res = []
    for text in data:
      res.append(text[:self.n])
    return th.LongTensor(res), labels

In [49]:
ND_cut = Truncate(df, n=5)
ND_cut[0], ND_cut[:3]

((tensor([[1860, 1482, 3704, 3817, 4105]]), 0),
 (tensor([[1860, 1482, 3704, 3817, 4105],
          [1718, 3608,  154, 3962, 5686],
          [1860, 4557, 1076, 4221, 5714]]),
  array([0, 1, 0], dtype=object)))

<p class="task" id="4"></p>

4\. Реализуйте преобразование `Pad`, которое расширяет каждый текст в батче до `n` символов значением `pad_idx`. Создайте версию `NewsDataset` с расширением предложений до 30 токенов. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [60]:
class Pad(NewsDataset):
  def __init__(self, data, n, pad_idx):
    super().__init__(data)
    self.n = n
    self.pad_idx = pad_idx

  def __getitem__(self, idx):
    data, labels = super().__getitem__(idx)
    res = []
    for text in data:
      text_pad = text.copy()
      while len(text_pad) < self.n:
        text_pad.append(self.pad_idx)
      res.append(text_pad)
    return th.LongTensor(res), labels

In [61]:
ND_pad = Pad(df, n=30, pad_idx=0)
print(ND_pad[0])
print(ND_pad[:3])

(tensor([[1860, 1482, 3704, 3817, 4105,  953,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]]), 0)
(tensor([[1860, 1482, 3704, 3817, 4105,  953,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [1718, 3608,  154, 3962, 5686, 3410, 1999, 1934,  683,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [1860, 4557, 1076, 4221, 5714, 3410, 1999, 1139, 2736, 2875,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]]), array([0, 1, 0], dtype=object))


<p class="task" id="5"></p>

5\. Создайте объект `torchtext.vocab.Vocab` на основе данных из файла `news.csv`. Первые 4 индекса зарезервируйте под специальные токены `<PAD>`, `<UNK>`, `<SOS>`, `<EOS>`. Опишите класс `NewsDatasetTorchText`, аналогичный по функционалу классу `NewsDataset`, но использующего реализацию `torchtext.vocab.Vocab`. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [2]:
df = pd.read_csv('news.csv')

In [3]:
corpus_tokens = df['text'].apply(lambda x: word_tokenize(x.lower()))
corpus_tokens.head(2)

0    [лукашенко, пригрозил, литовским, танкам, бело...
1    [российские, компании, оказались, в, опасности...
Name: text, dtype: object

In [4]:
vocab = build_vocab_from_iterator(corpus_tokens, specials=['<PAD>', '<UNK>', '<SOS>', '<EOS>'])

In [5]:
vocab.lookup_token(0)

'<PAD>'

In [30]:
class NewsDatasetTorchText:
  def __init__(self, data):
    self.data = data
    corpus_tokens = data['text'].apply(lambda x: word_tokenize(x.lower()))
    self.vcb = build_vocab_from_iterator(
        corpus_tokens,
        specials=['<PAD>', '<UNK>', '<SOS>', '<EOS>']
    )

  def __getitem__(self, idx):
    if isinstance(idx, slice):
      res = self.data.loc[idx.start:idx.stop:idx.step]
      tokens = res['text'].apply(lambda x: word_tokenize(x.lower()))
      enc_data = [self.vcb.lookup_indices(tok) for tok in tokens.values]
      return enc_data, res['label'].values
    else:
      res = self.data.loc[idx]
      tokens = word_tokenize(res['text'].lower())
      enc_data = self.vcb.lookup_indices(tokens) # тут можно навесить тензор, но я не буду, т.к. в последнем задании мне это не нужно
      return [enc_data], res[1]

In [14]:
NDTT = NewsDatasetTorchText(df)
NDTT[0], NDTT[:3]

((tensor([  31,  203, 3355, 5413,  491, 3095]), 0),
 ([[31, 203, 3355, 5413, 491, 3095],
   [27, 147, 3913, 4, 379, 12, 10, 2422, 4484],
   [31, 63, 29, 4036, 318, 12, 10, 3667, 568, 991],
   [5, 4955, 6, 728, 1127, 14, 3871, 1976, 4681, 3776]],
  array([0, 1, 0, 1])))

<p class="task" id="6"></p>

6\. Создайте преобразование, которое последовательно:
* преобразует набор токенов в последовательность индексов;
* преобразует результат в тензор;
* расширяет предложения до 30 символов, заполняя недостающие позиции индексом 0.

Создайте версию NewsDatasetTorchText с указанием этого преобразования. Выведите на экран результат выполнения `dataset[0]` и `dataset[:3]`

- [ ] Проверено на семинаре

In [38]:
class Transform(NewsDatasetTorchText):
  def __init__(self, data, n_pad, pad_idx=0):
    super().__init__(data)
    self.n = n_pad
    self.pad_idx = pad_idx

  def __getitem__(self, idx):
    data, labels = super().__getitem__(idx)
    res = []
    for text in data:
      text_pad = text.copy()
      while len(text_pad) < self.n:
        text_pad.append(self.pad_idx)
      res.append(text_pad)
    return th.LongTensor(res), labels

In [39]:
tr = Transform(df, 30)
tr[0]

(tensor([[  31,  203, 3355, 5413,  491, 3095,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0]]),
 0)

In [40]:
tr[:3]

(tensor([[  31,  203, 3355, 5413,  491, 3095,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [  27,  147, 3913,    4,  379,   12,   10, 2422, 4484,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [  31,   63,   29, 4036,  318,   12,   10, 3667,  568,  991,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0],
         [   5, 4955,    6,  728, 1127,   14, 3871, 1976, 4681, 3776,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0]]),
 array([0, 1, 0, 1]))

## Обратная связь
- [x] Хочу получить обратную связь по решению